In [1]:
#get_ipython().magic('reset -sf')

# Models

In [132]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import neighbors
from sklearn import feature_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
import time
import dalex as dx
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
from torch.autograd import Variable 
from captum.attr import (
    IntegratedGradients,
    GradientShap,
    DeepLift,
    DeepLiftShap,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)
import lime.lime_tabular

import matplotlib.pyplot as plt

First, I import pre-cleaned data, after I divide the dataset in training and testing, 60% and 40% respectively. Everything has to be scaled and I prepare the folds for the 5 cross validation. The steps I am going to follow in wach model are the following:
- Recursive feature elimination, I am going to remove the non important features
- Grid search, in order to try all the hyperparameters and choose the best ones
- Create model with the hyperparameters selected previously and fit it
- Make the predictions
- Shapley values to understand the model and the predictions

In [16]:
data_info_original = pd.read_csv('info_data.csv')
data_info_original = data_info_original.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info = pd.read_csv('info_datav2.csv')
data_info = data_info.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_gaussian = data_info[data_info.ANO_FACTURA == 2020]

In [127]:
def getISOCountry(iso_country):
    indexes = data_info_original[data_info_original['NUMERO_DEUDOR_PAIS_ID'] == iso_country].index
    iso_code = data_info.iloc[[indexes[0]]]['NUMERO_DEUDOR_PAIS_ID']
    return iso_code.values[0]

def getValuesFilter(ds, value, columns, target):
    if value == '*': #no filter 
        return ds
    indexes = getIndexFilter(ds, value, target)
    datat = getValues(ds, indexes, columns)
    return datat

def getIndexFilter(dt, value, target):
    indexes = dt[dt[target] == value].index
    return indexes

def getValues(dt, indexes, columns):
    datat = []
    for k, v in zip(dt.index, dt.values):
        if k in indexes:
            datat.append(v)
    df = pd.DataFrame(datat, columns = columns)
    return df


In [128]:
# Filter data by country
iso_spain = getISOCountry('ES') # Get iso from a country you want  to
data_info_filtered = getValuesFilter(data_info, '*', data_info.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by numero deudor pais id
data_info_filtered = data_info_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
data_info_original_filtered = getValuesFilter(data_info_original, '*', data_info_original.columns, 'NUMERO_DEUDOR_PAIS_ID') # iso = * -> no filter by numero deudor pais id
data_info_original_filtered = data_info_original_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])

In [19]:
traindataset_original, testdataset_original = train_test_split(data_info_original_filtered, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
traindataset, testdataset = train_test_split(data_info_filtered, test_size=0.4, shuffle= False) # To use all the data, change to -> data_info
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')

In [20]:
#scaler = StandardScaler()
#x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
#x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)
normalizer = MinMaxScaler(feature_range = (-1, 1))
x_train = pd.DataFrame(normalizer.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(normalizer.fit_transform(x_test), columns= x_test.columns)
mseresults = pd.DataFrame()
timeexecution = pd.DataFrame()

In [7]:
folds = KFold(n_splits = 5, shuffle = False) # if shuffle false, random state doesn't matter

## Linear Model

I have to choose which are the most important features in the model, I will use R2 to get it.

In [ ]:
hyper_params = [{'n_features_to_select': list(range(1, 26))}]
lm = LinearRegression()
rfe = RFE(lm)
model_cv = GridSearchCV(estimator = rfe, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv.fit(x_train, y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)

In [ ]:
display(cv_results)
print(model_cv.best_params_)

In [ ]:
#plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')
plt.savefig('../Output/testscoretrain.png')
plt.show()

display(cv_results[['param_n_features_to_select', 'mean_train_score', 'mean_test_score']])

In [9]:
n_features_optimal = 25 # Put the best parametrization Check if the previous RFE has the parameters -> Yes: Don't use this, take them directly
lm = LinearRegression()
rfe = RFE(lm, n_features_to_select= n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

I select only the parameters I am interested in

In [10]:
chosen = pd.DataFrame(rfe.support_, index=x_train.columns, columns=['Rank'])
featuresnotselected = []
for k, v in zip(chosen.index, chosen.values):
    if v == False:
        featuresnotselected.append(k)
x_train_final = x_train.drop(columns= featuresnotselected)
x_test_final = x_test.drop(columns= featuresnotselected)

Finally, I create the model with the best parametrization possible

In [11]:
lm = LinearRegression()
start_time = time.time()
lm.fit(x_train_final, y_train)
timeexecution['lm'] = (time.time() - start_time)
y_pred = lm.predict(x_test_final)
results = y_test.copy()
results['lm'] = y_pred

In [12]:
display(results)

#De moment no ho utilitzaré
F-test, p-values:

In [ ]:
f_val, p_val = feature_selection.f_regression(x_train_final, y_train) #Repassar
display(list(zip(x_train_final.columns, p_val)))
print('')
display(list(zip(x_train_final.columns, f_val)))

In [13]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test, y_pred))
mseresults['lm'] = metrics.mean_squared_error(y_test, y_pred)

In [ ]:
plt.barh(range(0, 25), lm.coef_[0])
plt.yticks(range(0, 25), x_train_final.columns, fontsize = 8)
plt.title('Coefficients')
plt.savefig('../Output/coefflinear.png')
plt.show()

In [ ]:
variable_groups = {
    'Date': ['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA', 'TEMPORADA_COMERCIAL_ID'],
    'Product': ['PRODUCTO_ID', 'TALLA', 'GRUPO_ARTICULO_PRODUCTO_ID', 'GENERO_PRODUCTO', 'CATEGORIA', 'TIPOLOGIA', 'CONSUMER_COLOR', 'CREMALLERA', 'CORDONES', 'OUTSOLE_SUELA_TIPO', 'OUTSOLE_SUELA_SUBTIPO', 'PLANTILLA_EXTRAIBLE'],
    'Age': ['EDAD_SN', 'EDAD_COMPRA'],
    'ContanctInfo': ['NUMERO_DEUDOR_PAIS_ID', 'CIUDAD_CONTACTO', 'IDIOMA_CONTACTO', 'GENERO_CONTACTO', 'CONTACTO_SN'],
    'SalePerson': ['ESFUERZO_VENTA_ID']
}

In [ ]:
linear_explainer = dx.Explainer(lm, x_test_final, y_test)
explanation = linear_explainer.model_parts()
explanation.plot()
groupedexpl = linear_explainer.model_parts(variable_groups=variable_groups)
groupedexpl.plot()

In [ ]:
bd_henry = linear_explainer.predict_parts(x_test_final.iloc[[20]], type = 'shap', B= 100)
bd_henry.plot()

In [ ]:
bd_henry = linear_explainer.predict_parts(x_test_final.iloc[[100]], type = 'shap', B= 100)
bd_henry.plot()

In [ ]:
display(testdataset.iloc[[100]])

## XGBoost Model

For this model, I am going to work with, the parameters max_depth, learning_rate and subsample from the Xgboost model and n_features_to_select from RFE.

In [ ]:
dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=list(x_train.columns))
dtest = xgb.DMatrix(x_test, feature_names=list(x_test.columns))

In [40]:
hyper_params = {'estimator__max_depth':[1, 10, 15], 'estimator__n_estimators': [150, 300, 500], 'estimator__learning_rate':[0.1, 0.05, 0.01] }#, 'n_features_to_select': list(range(1, 26))}
num_rounds = 1500
xgbm = xgb.XGBRegressor()
#rfe_xgboost = RFE(xgbm)
model_cv_xgb = GridSearchCV(estimator = xgbm, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv_xgb.fit(x_train, y_train)
cv_results = pd.DataFrame(model_cv.cv_results_)

Once I get the best parametrization, I execute the model with it, by the moment this is an example because I haven't execute the previous, too long.

In [23]:
xgbm = xgb.XGBRegressor(learning_rate =0.01, n_estimators=500, max_depth=15, min_child_weight=1, subsample=0.8, nthread=4, seed=39931191, verbose_eval = True)
start_time = time.time()
xgbm.fit(x_train, y_train)
timeexecution['xgboost'] = (time.time() - start_time)
y_pred = xgbm.predict(x_test)

In [ ]:
#results = y_test.copy()
results['xgboost'] = y_pred

In [24]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
mseresults['xgboost'] = metrics.mean_squared_error(y_test, y_pred)

Firstly, I create an explainer for the model, the inputs are the model, x_train and y_train. After, i can get the variable importance and the reverse cumulative distribution of residuals

In [28]:
explainer = dx.Explainer(xgbm, x_train, y_train)
explanation = explainer.model_parts()
explanation.plot()
print(explainer.model_performance().result)
res = explainer.model_performance(model_type='regression')
#res.plot()

Also, I can group the features into a new ones. In this case, I have created different variables that include a similar meaning.
- Date: ANO_FACTURA, MES_FACTURA, FECHA_FACTURA, TEMPORADA_COMERCIAL_ID
- Product: PRODUCTO_ID, TALLA, GRUPO_ARTICULO_PRODUCTO_ID, GENERO_PRODUCTO, CATEGORIA, TIPOLOGIA, CONSUMER_COLOR, CREMALLERA, CORDONES, OUTSOLE_SUELA_TIPO, OUTSOLE_SUELA_SUBTIPO, PLANTILLA_EXTRAIBLE
- Age: EDAD_SN, EDAD_COMPRA
- ContanctInfo: NUMERO_DEUDOR_PAIS_ID, CIUDAD_CONTACTO, IDIOMA_CONTACTO, GENERO_CONTACTO, CONTACTO_SN
- SalePerson: ESFUERZO_VENTA_ID

In [29]:
vars_grouped = explainer.model_parts(variable_groups= variable_groups)
print(vars_grouped.result)
vars_grouped.plot()

In [ ]:
sh = explainer.predict_parts(x_test.iloc[[20]], type = 'shap')
sh.plot()
display(testdataset.iloc[[20]])

In [ ]:
sh = explainer.predict_parts(x_test.iloc[[100]], type = 'shap')
sh.plot()
display(testdataset.iloc[[100]])

In [ ]:
#plt.figure(figsize=(16,6))
#chosen = pd.DataFrame(rfe_xgboost.support_, index=x_train.columns, columns=['Rank'])
#featuresnotselected = []
#for k, v in zip(chosen.index, chosen.values):
#    if v == False:
#        featuresnotselected.append(k)
#x_train_final = x_train.drop(columns= featuresnotselected)
#x_test_final = x_test.drop(columns= featuresnotselected)

#plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
#plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
#plt.xlabel('number of features')
#plt.ylabel('r-squared')
#plt.title("Optimal Number of Features")
#plt.legend(['test score', 'train score'], loc='upper left')

## k-nearest neighbour regression

In [ ]:
rmse_val = [] #to store rmse values for different k
mse_val = []
model = neighbors.KNeighborsRegressor(n_neighbors = 7)
start_time = time.time()
model.fit(x_train, y_train)  #fit the model
timeexecution['knn'] = (time.time() - start_time)
y_pred=model.predict(x_test) #make prediction on test set
mse = metrics.mean_squared_error(y_test, y_pred)
error = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) #calculate rmse
rmse_val.append(error) #store rmse values
mse_val.append(mse)
results['knn'] = y_pred
mseresults['knn'] = metrics.mean_squared_error(y_test, y_pred)

In [ ]:
#curvermse = pd.DataFrame(rmse_val) #elbow curve 
#curvermse.plot()
#curvemse = pd.DataFrame(mse_val)
#curvemse.plot()

## NN

In [25]:
nn_reg = MLPRegressor(hidden_layer_sizes=(300, 300),  activation='logistic', solver='adam', alpha=0.01, batch_size='auto', learning_rate='constant', learning_rate_init=0.01, max_iter=1000, shuffle=False, tol=0.0001, verbose=True, early_stopping= True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nn_reg.fit(x_train, np.ravel(y_train))
y_pred = nn_reg.predict(x_test)
nn_reg.score(x_test, y_test)

In [26]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))

In [75]:
nn_reg_pred = lambda x: nn_reg.predict(x).astype(float)

In [ ]:
results['nn'] = y_pred

In [130]:
indexspaindata = getIndexFilter(testdataset, 14, 'NUMERO_DEUDOR_PAIS_ID')
indexspaindata_original = getIndexFilter(testdataset_original, 'ES', 'NUMERO_DEUDOR_PAIS_ID')
print(indexspaindata)
print(indexspaindata_original)

In [123]:
display(testdataset.loc[indexspaindata[0]])
display(testdataset_original.loc[indexspaindata_original[0]])
display(testdataset_original[(testdataset_original['PRODUCTO_ID'] == 10189) & (testdataset_original['NUMERO_DEUDOR_PAIS_ID'] == 'ES') & (test)].index)
display(testdataset_original.loc[[644943]])
i=0
for v in indexspaindata_original:
    if v == 644943:
        break
    else:
        i = i+1
print(i)


In [101]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=x_train.columns, verbose=True, mode='regression')
exp = explainer.explain_instance(np.ravel(x_test.iloc[[indexspaindata[0]]]), nn_reg_pred, num_features=10)
exp.show_in_notebook(show_table=True)
exp.as_pyplot_figure()

In [102]:
exp.as_list()

In [68]:
display(testdataset.iloc[[100]])
display(y_pred[100])

## LSTM

In [134]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [167]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.01 #0.001 lr

input_size = 25 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes

x_train_tensors = Variable(torch.Tensor(x_train.values))
x_test_tensors = Variable(torch.Tensor(x_test.values))
y_train_tensors = Variable(torch.Tensor(y_train.values))
y_test_tensors = Variable(torch.Tensor(y_test.values))

x_train_tensors_final = torch.reshape(x_train_tensors,   (x_train_tensors.shape[0], 1, x_train_tensors.shape[1]))


x_test_tensors_final = torch.reshape(x_test_tensors,  (x_test_tensors.shape[0], 1, x_test_tensors.shape[1]))

#ig = IntegratedGradients(nn_reg)
#input = torch.randn(, requires_grad=True)
#t = torch.Tensor(x_test.iloc[[20]].values)
# Computes integrated gradients for class 3.
#attribution = ig.attribute(t, target=3)
x_train_tensors.shape

In [168]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, x_train_tensors_final.shape[1]) #our lstm class

In [169]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)

In [150]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(x_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

In [166]:
train_predict = lstm1(x_test_tensors_final)#forward pass
data_predict = train_predict.data.numpy() #numpy conversion
#plt.axvline(x=200, c='r', linestyle='--') #size of the training set
print(y_test)
plt.plot(range(0, 3),y_test[:3], label='Actuall Data') #actual plot
plt.plot(range(0, 3), data_predict[:3], label='Predicted Data') #predicted plot
plt.title('Time-Series Prediction')
plt.legend()
plt.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, data_predict))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, data_predict))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, data_predict)))
print('R2:', metrics.r2_score(y_test, data_predict))

## Gaussian process regression

In [ ]:
#params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

#knn = neighbors.KNeighborsRegressor()

#model = GridSearchCV(knn, params, cv=10)
#model.fit(x_train,y_train)
#model.best_params_

In [22]:
data_gaussian_filtered = getValuesFilter(data_gaussian, 14, data_gaussian.columns) # iso = * -> no filter by numero deudor pais id
data_gaussian_filtered = data_gaussian_filtered.sort_values(['ANO_FACTURA', 'MES_FACTURA', 'FECHA_FACTURA'])
traindataset, testdataset = train_test_split(data_gaussian, test_size=0.01, shuffle= False)
traindataset, testdataset = train_test_split(testdataset, test_size=0.01, shuffle= False)
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])
x_test = testdataset.loc[:, testdataset.columns != 'IMP_VENTA_NETO_EUR']
y_test = testdataset.loc[:, testdataset.columns == 'IMP_VENTA_NETO_EUR']
x_test = x_test.drop(columns = 'EDAD_RANGO_COMPRA')
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)
traindataset.shape

In [23]:
# massive dataset, try with something smaller
kernel = kernel = RBF(10, (1e-2, 1e2)) # Check which ones choose
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gpr.fit(x_train, y_train)
gpr.score(x_train, y_train)
y_pred, sigma = gpr.predict(x_test, return_std=True)
results['gaussianP'] = y_pred

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))

## Lasso Regression

In [ ]:
hyper_params = {'alpha':[1, 0.5, 0.1, 0.01, 0.001]}
num_rounds = 1500
lasso = Lasso()
model_cv_lasso = GridSearchCV(estimator = lasso, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1, return_train_score=True)      
model_cv_lasso.fit(x_train, y_train)
cv_results = pd.DataFrame(model_cv_lasso.cv_results_)

In [ ]:
model_cv_lasso.best_params_

In [ ]:
lasso = Lasso(alpha= 0.01)
start_time = time.time()
model = lasso.fit(x_train, y_train)
timeexecution['lasso'] = (time.time() - start_time)
y_pred = lasso.predict(x_test)

In [ ]:
#results = y_test.copy()
results['lasso'] = y_pred

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
mseresults['lasso'] = metrics.mean_squared_error(y_test, y_pred)

In [ ]:
explainer = dx.Explainer(lasso, x_train, y_train)
explanation = explainer.model_parts()
explanation.plot()
print(explainer.model_performance().result)
res = explainer.model_performance(model_type='regression')

In [ ]:
vars_grouped = explainer.model_parts(variable_groups= variable_groups)
print(vars_grouped.result)
vars_grouped.plot()

In [ ]:
sh = explainer.predict_parts(x_test.iloc[[20]], type = 'shap')
sh.plot()
display(testdataset.iloc[[20]])

In [ ]:
sh = explainer.predict_parts(x_test.iloc[[100]], type = 'shap')
sh.plot()
display(testdataset.iloc[[100]])

## Decision Tree

In [ ]:
rfr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=20)
start_time = time.time()
model = rfr.fit(X, y)
timeexecution['RandomForest'] = (time.time() - start_time)
y_pred = rfr.predict(x_test)
results['RandomForest'] = y_pred